## Homegate Analysis

In [9]:
# Preliminary
import pandas as pd
import os
import numpy as np
os.chdir('/Users/jindi/Documents/GitHub/pp4r_final_assignments') # change this to your path

In [10]:
# Read file
df = pd.read_csv('data/Homegate_data_cleaned.csv')
df.head()

,Net Rent (CHF),Additional Expenses (CHF),Gross Rent (CHF),Available Date,Property Type,Number of Rooms,Number of Toilets,Floor Level,Living Area (sq. m.),Year Built,...,Room Height (m.),Last Refurbishment Year,Number of Floors,Floor Space (sq. m.),Land Area (sq. m.),Number of Apartments,Property Volume (cubic m.),Street,Postcode,City
0,3015.0,380.0,3395.0,01.11.2023,Apartment,3.5,1.0,6.0,88.0,2015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfingstweidstrasse 106b,8005.0,Zürich
1,3050.0,400.0,3450.0,01.12.2023,Apartment,2.5,NaN,1.0,80.0,2017.0,...,2.5,NaN,NaN,NaN,NaN,NaN,NaN,Gläserenstrasse 14,8142.0,Uitikon Waldegg
2,2445.0,323.0,2768.0,Immediately,Apartment,4.5,NaN,4.0,100.0,NaN,...,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,Sihlfeldstrasse 89,8004.0,Zürich
3,3235.0,260.0,3495.0,01.11.2023,Apartment,4.0,NaN,2.0,95.0,1921.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kreuzstrasse 6,8802.0,Kilchberg
4,NaN,NaN,NaN,01.10.2023,Apartment,4.5,NaN,24.0,127.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hagenholzstrasse,8050.0,Zürich


### Geographical Analysis 

Here we can map the listings we have to a map in Zurich? (since we do not have the geographical coordinates yet, we can just plot by postcode?)

### Plots

### Regressions 